In [116]:
# %pip install numpy
# %pip install pandas
import numpy as np
import pandas as pd

Put copy and paste into chatgpt with this prompt:
Put this into a csv 
the column for score (i.e. 3 / 3) should be exactly (3 / 3), not 1 or simplified at all
The names of the columns should be: GRADE_ITME, POINTS
Delete rows that do not contain any numbers

In [117]:

df = pd.read_csv('DaltonGrades.csv')
# df = pd.read_csv('kylie.csv')
# df = pd.read_csv('luke.csv')

# rename columns from format Grade Item to GRADE_ITEM
df.columns = df.columns.str.replace(' ', '_').str.upper()

# remove all rows where GRADE_ITEM is null 
df = df[df['GRADE_ITEM'].notnull()]

# rename each row in GRADE_ITEM to from format In-Class 1 to IN_CLASS_1
df['GRADE_ITEM'] = df['GRADE_ITEM'].str.replace(' ', '_').str.upper()
df['GRADE_ITEM'] = df['GRADE_ITEM'].str.replace('-', '_').str.upper()

# REMOVE ALL # IN GRADE_ITEM
df['GRADE_ITEM'] = df['GRADE_ITEM'].str.replace('#', '')

,GRADE_ITEM,POINTS,GRADE,COMMENTS_AND_ASSESSMENTS
0,WEBWORK,80 / 100,80%,NaN
1,IN_CLASS_1,4.5 / 6,75%,NaN
2,IN_CLASS_2,6 / 7,85.71%,NaN
3,IN_CLASS_3,8 / 9,88.89%,NaN
4,WRITTEN_HOMEWORK_1,10 / 13,76.92%,NaN


In [118]:
# seperate "POINTS_EARNED" and "POINTS_POSSIBLE" into seperate columns
df[['POINTS_EARNED', 'POINTS_POSSIBLE']] = df['POINTS'].str.split('/', expand=True)

# create column ITEM_TYPE based off of GRADE_ITEM being either IN_CLASS, HOMEWORK, or EXAM
df['ITEM_TYPE'] = np.where(df['GRADE_ITEM'].str.contains('IN_CLASS'), 'IN_CLASS', 
                           np.where(df['GRADE_ITEM'].str.contains('HOMEWORK'), 'HOMEWORK', 
                                    np.where(df['GRADE_ITEM'].str.contains('EXAM'), 'EXAM', 'HOMEWORK')))

# COMPUTE GRADE VIA POITNS
df['GRADE'] = (df['POINTS_EARNED'].astype(float) / df['POINTS_POSSIBLE'].astype(float)) * 100

df.sort_values(by=['ITEM_TYPE', 'GRADE_ITEM'], inplace=True)
df

,GRADE_ITEM,POINTS,GRADE,COMMENTS_AND_ASSESSMENTS,POINTS_EARNED,POINTS_POSSIBLE,ITEM_TYPE
9,EXAM_1,64.5 / 86,75.000000,NaN,64.5,86,EXAM
20,EXAM_2,55 / 89,61.797753,NaN,55,89,EXAM
29,FINAL_PROPOSAL,12 / 12,100.000000,NaN,12,12,HOMEWORK
17,LAB_1,0 / 10,0.000000,Overall Feedback,0,10,HOMEWORK
0,WEBWORK,80 / 100,80.000000,NaN,80,100,HOMEWORK
4,WRITTEN_HOMEWORK_1,10 / 13,76.923077,NaN,10,13,HOMEWORK
8,WRITTEN_HOMEWORK_2,10 / 12,83.333333,NaN,10,12,HOMEWORK
14,WRITTEN_HOMEWORK_3,6 / 11,54.545455,NaN,6,11,HOMEWORK
27,WRITTEN_HOMEWORK_4,16 / 21,76.190476,NaN,16,21,HOMEWORK
1,IN_CLASS_1,4.5 / 6,75.000000,NaN,4.5,6,IN_CLASS


In [119]:
# filter down to just in class
in_class_df = df[df['ITEM_TYPE'] == 'IN_CLASS'].sort_values(by='GRADE', ascending=False)
# select all but the 3 lowest grades
in_class_df = in_class_df.iloc[:-3]

in_class_earned = in_class_df['POINTS_EARNED'].astype(float).sum()
in_class_possible = in_class_df['POINTS_POSSIBLE'].astype(float).sum()
in_class_grade = (in_class_earned / in_class_possible) * 100
in_class_grade

93.31683168316832

In [120]:
# filter down to just hw
hw_df = df[df['ITEM_TYPE'] == 'HOMEWORK'].sort_values(by='GRADE', ascending=False)

hw_possible = hw_df['POINTS_POSSIBLE'].astype(float).sum()
hw_earned = hw_df['POINTS_EARNED'].astype(float).sum()

hw_grade = (hw_earned / hw_possible) * 100
hw_grade

74.86033519553072

In [121]:
# filter to exams
exam_df = df[df['ITEM_TYPE'] == 'EXAM'].sort_values(by='GRADE', ascending=False)

exam_possible = exam_df['POINTS_POSSIBLE'].astype(float).sum()
exam_earned = exam_df['POINTS_EARNED'].astype(float).sum()

exam_grade = (exam_earned / exam_possible) * 100
exam_grade

68.28571428571428

In [123]:
# total_grade is just all the earned points divided by all possible points for every category
total_grade = ((in_class_earned + hw_earned + exam_earned) / (in_class_possible + hw_possible + exam_possible)) * 100

In [124]:
# print results
print('In Class Grade: ' + str(in_class_grade))
print('Homework Grade: ' + str(hw_grade))
print('Exam Grade: ' + str(exam_grade))
print('Total Grade: ' + str(total_grade))

In Class Grade: 93.31683168316832
Homework Grade: 74.86033519553072
Exam Grade: 68.28571428571428
Total Grade: 76.42857142857142
